In [1]:
#Importar las librerias que vamos a utilizar
import glob
import pandas as pd
import numpy as np
import os
import xgboost as xgb
from sklearn.model_selection import GroupShuffleSplit

In [2]:
#Añadir todos los csv
archivos_csv = glob.glob('../Caballos/ID-Carreras/*.csv') #('../PRACTICAS/Programa de carreras/CSV/*.csv')
print(archivos_csv)

['../Caballos/ID-Carreras\\1_01-01-2017.csv', '../Caballos/ID-Carreras\\1_01-06-2014.csv', '../Caballos/ID-Carreras\\1_01-07-2021.csv', '../Caballos/ID-Carreras\\1_01-08-2013.csv', '../Caballos/ID-Carreras\\1_01-11-2015.csv', '../Caballos/ID-Carreras\\1_01-12-2013.csv', '../Caballos/ID-Carreras\\1_02-07-2011.csv', '../Caballos/ID-Carreras\\1_02-10-2011.csv', '../Caballos/ID-Carreras\\1_02-11-2014.csv', '../Caballos/ID-Carreras\\1_02-12-2012.csv', '../Caballos/ID-Carreras\\1_03-04-2016.csv', '../Caballos/ID-Carreras\\1_03-06-2012.csv', '../Caballos/ID-Carreras\\1_03-07-2016.csv', '../Caballos/ID-Carreras\\1_03-07-2020.csv', '../Caballos/ID-Carreras\\1_03-08-2012.csv', '../Caballos/ID-Carreras\\1_03-10-2014.csv', '../Caballos/ID-Carreras\\1_04-05-2012.csv', '../Caballos/ID-Carreras\\1_04-05-2014.csv', '../Caballos/ID-Carreras\\1_04-08-2016.csv', '../Caballos/ID-Carreras\\1_04-09-2011.csv', '../Caballos/ID-Carreras\\1_04-10-2015.csv', '../Caballos/ID-Carreras\\1_04-12-2011.csv', '../Cabal

In [3]:
# UNIR TODAS LAS CARRERAS

# creamos un primer dataframe y lo eliminamos del conjunto
filename = archivos_csv[0]
archivos_csv.remove(filename)
df = pd.read_csv(filename, delimiter=',',encoding="utf-8",header=0)

# Escribimos un loop que irá a través de cada uno de los nombres de archivo a través de globbing y el resultado final será el dataframe final

for filename in archivos_csv:
    data = pd.read_csv(filename, delimiter=',',encoding="utf-8",header=0)
    df = df.append(data)


#Para chequear que todo está bien, mostramos la lista por consola
#df



In [4]:
'''
SEPARAR LOS DATOS POR CARRERAS
#Agrupar los datos
x_head = ['ID','CABALLO','POSICION','EDAD','KG','JINETE','CUADRA','PREPARADOR','CAJÓN','HIPODROMO','DISTANCIA','PISTA']


for filename in archivos_csv:
    data = pd.read_csv(filename, delimiter=';',encoding="utf-8",header=None)
    
    data.columns = x_head
    
    grouped = data.groupby(x_head[0])
    
    for value, group in grouped:
        filename= str(value)+'.csv'
        try: 
            f = open(filename,'w')
            group.to_csv(filename, header=x_head,index=False)
            print(filename)
        except UnicodeEncodeError:
            print('Error de codificacion')
'''


'\nSEPARAR LOS DATOS POR CARRERAS\n#Agrupar los datos\nx_head = [\'ID\',\'CABALLO\',\'POSICION\',\'EDAD\',\'KG\',\'JINETE\',\'CUADRA\',\'PREPARADOR\',\'CAJÓN\',\'HIPODROMO\',\'DISTANCIA\',\'PISTA\']\n\n\nfor filename in archivos_csv:\n    data = pd.read_csv(filename, delimiter=\';\',encoding="utf-8",header=None)\n    \n    data.columns = x_head\n    \n    grouped = data.groupby(x_head[0])\n    \n    for value, group in grouped:\n        filename= str(value)+\'.csv\'\n        try: \n            f = open(filename,\'w\')\n            group.to_csv(filename, header=x_head,index=False)\n            print(filename)\n        except UnicodeEncodeError:\n            print(\'Error de codificacion\')\n'

In [5]:
#Vamos a comprobar que tipo de datos tiene cada columna
df.dtypes

ID             object
CABALLO        object
POSICION       object
EDAD           object
KG             object
JINETE         object
CUADRA         object
PREPARADOR     object
CAJÓN         float64
HIPODROMO      object
DISTANCIA       int64
PISTA          object
dtype: object

In [6]:
df['ID'].value_counts()

3_04-12-2011    20
3_17-09-2017    20
2_30-09-2012    20
3_30-09-2012    20
3_17-11-2013    20
                ..
3_26-09-2014     4
2_25-05-2014     4
5_27-08-2012     4
1_3-12-2017      4
5_17-06-2020     4
Name: ID, Length: 1399, dtype: int64

In [7]:
#Comprobamos que los datos de las columnas estan correctas
df['EDAD'].value_counts().head(50)

#Podemos observar que hay datos de KG en EDAD y que hay diferentes formatos para definir un numero

3 años     2051
4 años     1324
3          1249
4          1229
5           861
3años      834
5 años      718
2 años      692
6           547
4años      478
6 años      433
2           381
5años      276
7           275
2años      205
7 años      197
8           184
6años      156
8 años       89
7años       71
9            64
2 aÃ±os      55
3 aÃ±os      52
8años       47
57           47
58           47
56           47
59           46
10           44
62           38
57           37
4 aÃ±os      37
60           36
55           33
54           33
9 años       32
61           31
56           24
58           21
9años       20
57,5         20
5 aÃ±os      18
55,5         18
59           17
11           17
10años      15
54,5         14
53           12
59,5         12
55           11
Name: EDAD, dtype: int64

In [8]:
#Ponemos en un solo formato
df = df.replace({"2 años": 2,"3 años":3,"4 años": 4,"5 años": 5,"6 años": 6,"7 años": 7,"8 años": 8,"9 años": 9,
                 "10 años": 10,"11 años": 11,"12 años": 12,"2años": 2,"3años": 3,"4años": 4, "5años": 5, "6años": 6,
                 "7años": 7,"8años": 8,"9años": 9,"10años": 10,"11años": 11,"12años": 12,"2 aÃ±os": 2,"3 aÃ±os": 3,
                 "4 aÃ±os": 4,"5 aÃ±os": 5,"6 aÃ±os": 6,"7 aÃ±os": 7,"8 aÃ±os": 8,"2:00 AM": 2,"3:00 AM": 3,"4:00 AM": 4,
                 "5:00 AM": 5,"6:00 AM": 6,"7:00 AM": 7,"8:00 AM": 8})

#Mostramos
df['EDAD'].value_counts().head(50)

3       4191
4       3078
5       1878
2       1340
6       1152
7        549
8        326
9        116
10        65
58        47
56        47
57        47
59        46
62        38
57        37
60        36
54        33
55        33
61        31
56        24
11        24
58        21
57,5      20
55,5      18
59        17
54,5      14
53        12
59,5      12
55        11
52        10
60        10
50        10
56,5       9
12         8
63         8
58,5       7
51         6
65         6
61         6
53,5       5
62         5
64,5       3
64         3
62,5       3
60,5       3
61,5       2
66         2
54         2
52,5       2
50         1
Name: EDAD, dtype: int64

In [9]:
#Hacemos exactamente lo mismo con todas las columnas y modificamos valores si es preciso
df['POSICION'].value_counts().head(50)

1.0       1399
2.0       1395
3.0       1395
4.0       1392
5.0       1354
6.0       1242
7.0       1118
8.0        930
9.0        754
99.0       674
10.0       593
11.0       452
12.0       289
13.0       128
14.0        87
15.0        54
16.0        33
17.0        23
18.0        12
19.0         7
20.0         3
33.0         2
6            2
2            2
7            2
8            2
4            2
3            2
1            1
10           1
01-ene       1
0            1
9            1
11           1
99           1
05-ene       1
5            1
Name: POSICION, dtype: int64

In [10]:
df.POSICION = df.POSICION.replace({1:1.0,2:2.0,3:3.0,4:4.0,5:5.0,
                                   33.0:3,"05-ene": 5,"01-ene": 1, "0.1": 1})
#df.loc[df['POSICION'] == 1, 'POSICION'] = 1

df['POSICION'].value_counts().head(50)

1.0     1400
3.0     1397
2.0     1395
4.0     1392
5.0     1355
6.0     1242
7.0     1118
8.0      930
9.0      754
99.0     674
10.0     593
11.0     452
12.0     289
13.0     128
14.0      87
15.0      54
16.0      33
17.0      23
18.0      12
19.0       7
20.0       3
2          2
7          2
8          2
6          2
4          2
3          2
1          1
10         1
0          1
9          1
11         1
99         1
5          1
Name: POSICION, dtype: int64

In [11]:
#Algunos datos en vez de espacios tienen un caracter raro, vamos a poner el espacio en su lugar
df.replace('',' ', regex=True, inplace=True)

In [12]:
#Mostrar las filas en las que no tenemos informacion de los caballos
df.loc[df['POSICION']==0.0]

,ID,CABALLO,POSICION,EDAD,KG,JINETE,CUADRA,PREPARADOR,CAJÓN,HIPODROMO,DISTANCIA,PISTA
0,5_23-04-2017,AJMANY (IRE),0,7,58,B.FAYOS,CHEIK ABDULLAH BIN KHALIFA AL THANI,Y.DUREPAIRE,8.0,Hipódromo de La Zarzuela,2000,HIERBA


In [13]:
#Una vez comprobadas las posiciones las insertamos en su lugar correspondiente
df.loc[df['POSICION']==0.0, 'POSICION'] = 99.0
df['POSICION'].value_counts().head(50)

1.0     1400
3.0     1397
2.0     1395
4.0     1392
5.0     1355
6.0     1242
7.0     1118
8.0      930
9.0      754
99.0     675
10.0     593
11.0     452
12.0     289
13.0     128
14.0      87
15.0      54
16.0      33
17.0      23
18.0      12
19.0       7
20.0       3
2          2
7          2
8          2
3          2
4          2
6          2
99         1
11         1
9          1
1          1
10         1
5          1
Name: POSICION, dtype: int64

In [14]:
df['KG'].value_counts().head(50)

57      1007
56       852
57       799
58       624
56       599
54,5     568
55,5     559
62       496
59       452
56,5     410
58,5     397
60       387
58       371
55       358
61       342
57,5     310
59       304
55       277
62       272
60       270
59,5     256
60,5     238
61       237
53,5     231
54       230
3        216
61,5     202
54       188
4        173
53       127
51       124
53       112
5        110
52,5     109
50       103
52        91
63        76
51        74
2         72
52        71
62,5      71
64        67
65        61
50        55
63,5      53
6         47
51,5      38
63        33
64,5      30
66        25
Name: KG, dtype: int64

In [15]:
#Los datos que hemos recogidos tienen una ',' como indicador de decimal y necesitamos un '.' para convertirlos a float
df.replace(',','.', regex=True, inplace=True)

#Convertimos los datos en el tipo que necesitamos
df['EDAD']=pd.to_numeric(df['EDAD'])
df['KG']=pd.to_numeric(df['KG'])
df['POSICION']=pd.to_numeric(df['POSICION'])

#Mostramos que esten correctos
df.dtypes

ID             object
CABALLO        object
POSICION      float64
EDAD          float64
KG            float64
JINETE         object
CUADRA         object
PREPARADOR     object
CAJÓN         float64
HIPODROMO      object
DISTANCIA       int64
PISTA          object
dtype: object

In [16]:
#Algunos datos de EDAD y KG estan intercambiados por lo que vamos a colocarlos adecuadamente
#Guardamos los datos que no estan bien en aux
auxEdad = df.loc[df['EDAD']>=20]
auxEdad = auxEdad['EDAD']

auxKG = df.loc[df['KG']<=20]
auxKG = auxKG['KG']

#Introducimos los datos en las columnas correspondientes
df.loc[df.EDAD >= 20, ['EDAD']] = auxKG
df.loc[df.KG <= 20, ['KG']] = auxEdad

In [17]:
#Comprobamos que se hayan ordenado correctamente
df['EDAD'].value_counts().head(50)


3.0     4407
4.0     3251
5.0     1988
2.0     1412
6.0     1199
7.0      571
8.0      333
9.0      120
10.0      65
11.0      24
12.0       8
14.0       1
Name: EDAD, dtype: int64

In [18]:
df['KG'].value_counts().head(50)

57.0    1890
56.0    1522
58.0    1063
59.0     819
62.0     811
60.0     703
55.0     679
61.0     616
54.5     582
55.5     577
54.0     453
56.5     419
58.5     404
57.5     330
59.5     268
53.0     251
60.5     241
53.5     236
61.5     204
51.0     204
52.0     173
50.0     169
63.0     118
52.5     111
64.0      90
65.0      77
62.5      74
63.5      53
51.5      38
66.0      36
64.5      33
67.0      30
65.5      26
68.0      19
66.5      18
50.5      13
69.0       7
67.5       7
70.0       6
68.5       2
71.0       2
74.0       2
71.5       1
72.5       1
69.5       1
72.0       1
Name: KG, dtype: int64

In [19]:
df['CAJÓN'].value_counts().head(50)

2.0     1399
1.0     1399
4.0     1399
3.0     1398
5.0     1385
6.0     1293
7.0     1162
8.0      999
9.0      833
10.0     688
11.0     532
12.0     421
13.0     158
14.0     114
15.0      77
16.0      46
17.0      34
18.0      22
19.0      14
20.0       6
Name: CAJÓN, dtype: int64

In [20]:
#Vamos a poner el hipodromo en mayusculas para que no haya problema entra mayusculas y minusculas
df['HIPODROMO'] = df['HIPODROMO'].str.upper()

In [21]:
pd.value_counts(df['HIPODROMO'])

HIPÓDROMO LA ZARZUELA                                            4203
HIPÓDROMO DE LA ZARZUELA                                         2946
GRAN HIPÓDROMO DE ANDALUCÍA JAVIER PIÑAR HAFNER                  1499
HIPODROMO DE SAN SEBASTIAN                                       1435
HIPÓDROMO DE SAN SEBASTIÁN                                        745
HIPÓDROMO COSTA DEL SOL.                                          603
GRAN HIPÓDROMO DE ANDALUCÍA                                       314
HIPODROMO LA ZARZUELA                                             299
GRAN HIPÓDROMO DE ANDALUCIA                                       233
GRAN HIPÓDROMO DE ANDALUCÍA JAVIER PIÑAR HAFNER-PARTICIPANTES     193
HIPÃ³DROMO LA ZARZUELA                                            178
HIPÓDROMO COSTA DE SOL                                            120
 HIPÓDROMO DE LA ZARZUELA                                          96
HIPÓDROMO DE LA ZARZUELA                                           57
GRAN HIPODROMO DE AN

In [22]:
#VAMOS A PONER EN UN SOLO FORMATO LOS DIFERENTES TIPOS DE HIPODROMO
df['HIPODROMO']=df['HIPODROMO'].replace(['HIPÓDROMO LA ZARZUELA', 'HIPÓDROMO DE LA ZARZUELA', 'HIPODROMO LA ZARZUELA', 
                         ' HIPÓDROMO DE LA ZARZUELA', 'HIPÃ³DROMO LA ZARZUELA', ' HIPÓDROMO DE LA ZARZUELA',
                         'HIPÓDROMO DE LA ZARZUELA ','HIPÓDROMO LA ZARZUELA '
                        ], 'HIPODROMO LA ZARZUELA')
df['HIPODROMO']=df['HIPODROMO'].replace(['GRAN HIPÓDROMO DE ANDALUCÍA JAVIER PIÑAR HAFNER',
                                         'GRAN HIPÓDROMO DE ANDALUCÍA','GRAN HIPÓDROMO DE ANDALUCIA',
                                         'GRAN HIPÓDROMO DE ANDALUCÍA JAVIER PIÑAR HAFNER-PARTICIPANTES',
                                         'GRAN HIPÃ³DROMO DE ANDALUCÃ­A JAVIER PIÃ±AR HAFNER-PARTICIPANTES',
                                         'GRAN HIPÓDROMO DE ANDALUCIA JAVIER PIÑAR HAFNER',
                                         'HIPÓDROMO DE ANDALUCÍA','GRAN HIPODROMO DE ANDALUCIA',
                                         ' GRAN HIPÓDROMO DE ANDALUCÍA'
                        ], 'GRAN HIPODROMO DE ANDALUCIA JAVIER PIÑAR HAFNER')
df['HIPODROMO']=df['HIPODROMO'].replace(['HIPÓDROMO DE SAN SEBASTIÁN','HIPÓDROMO DE SAN SEBASTIAN',
                                        ' HIPÓDROMO DE SAN SEBASTIÁN'
                        ], 'HIPODROMO DE SAN SEBASTIAN')
df['HIPODROMO']=df['HIPODROMO'].replace(['HIPÓDROMO COSTA DEL SOL.','HIPÓDROMO COSTA DE SOL',
                                        'HIPÃ³DROMO COSTA DEL SOL.','HIPÓDROMO COSTA DEL SOL','HIPÓDROMO DE LA COSTA DEL SOL '
                        ], 'HIPODROMO COSTA DEL SOL')
pd.value_counts(df['HIPODROMO'])

HIPODROMO LA ZARZUELA                              7830
GRAN HIPODROMO DE ANDALUCIA JAVIER PIÑAR HAFNER    2402
HIPODROMO DE SAN SEBASTIAN                         2238
HIPODROMO COSTA DEL SOL                             869
HIPÓDROMO DE PINEDA                                  41
Name: HIPODROMO, dtype: int64

In [23]:
df

,ID,CABALLO,POSICION,EDAD,KG,JINETE,CUADRA,PREPARADOR,CAJÓN,HIPODROMO,DISTANCIA,PISTA
0,1_01-01-2017,SURFINA (GB),1.0,3.0,56.5,R. SOUSA,ATHEIRAM,O. ANAYA,1.0,GRAN HIPODROMO DE ANDALUCIA JAVIER PIÑAR HAFNER,1300,HIERBA
1,1_01-01-2017,BEAR ESSENTIALS (GB),3.0,3.0,56.0,A. GOMEZ,ROCCO'S,T. MARTINS,4.0,GRAN HIPODROMO DE ANDALUCIA JAVIER PIÑAR HAFNER,1300,HIERBA
2,1_01-01-2017,ORANGE GIN (IRE),6.0,3.0,56.0,J.E. JARCOVSKY (57),CELSO MENDEZ,M. ALVAREZ,5.0,GRAN HIPODROMO DE ANDALUCIA JAVIER PIÑAR HAFNER,1300,HIERBA
3,1_01-01-2017,DREAMS O'CONNOR (GB),2.0,3.0,54.5,A. MEZZATESTA,SCUDERIA DREAMS ROAD,M. AUGELLI,2.0,GRAN HIPODROMO DE ANDALUCIA JAVIER PIÑAR HAFNER,1300,HIERBA
4,1_01-01-2017,ENSUEÑO,4.0,3.0,54.5,E.J. ARGUINZONES,CIELO DE MADRID,J.A. RODRIGUEZ,3.0,GRAN HIPODROMO DE ANDALUCIA JAVIER PIÑAR HAFNER,1300,HIERBA
...,...,...,...,...,...,...,...,...,...,...,...,...
7,7_26-10-2014,FIGUEIRAS,3.0,4.0,56.0,J.GELABERT (53),PEREIRA S.A.T.,CALLEJO S.. T,8.0,HIPODROMO LA ZARZUELA,1800,HIERBA
8,7_26-10-2014,SUEVA,2.0,3.0,55.5,J.CROCQUEVIEILLE,TIZIANO,MAROTO. J.L.,10.0,HIPODROMO LA ZARZUELA,1800,HIERBA
9,7_26-10-2014,AHORA NO (GB),4.0,5.0,54.0,V.JANACEK (55),JM,CARRASCO. A.,1.0,HIPODROMO LA ZARZUELA,1800,HIERBA
10,7_26-10-2014,SINNIEVA (FR),6.0,4.0,52.0,M.SECCI (52.5),ZURRAQUIN,HERNANDEZ. M.,4.0,HIPODROMO LA ZARZUELA,1800,HIERBA


In [24]:
#Con XGBoots no podemos tener datos categoricos, por lo que vamos a realizar una codificacion en caliente
df = pd.get_dummies(df,columns=['CABALLO','JINETE','CUADRA','PREPARADOR','HIPODROMO','PISTA'])
df

,ID,POSICION,EDAD,KG,CAJÓN,DISTANCIA,CABALLO_AARASH,CABALLO_AARHUS,CABALLO_AAWAM DE GRINE (MOR),CABALLO_ABANTOS (IRE),...,PREPARADOR_Y. DUREPAIRE,PREPARADOR_Y.DUREPAIRE,HIPODROMO_GRAN HIPODROMO DE ANDALUCIA JAVIER PIÑAR HAFNER,HIPODROMO_HIPODROMO COSTA DEL SOL,HIPODROMO_HIPODROMO DE SAN SEBASTIAN,HIPODROMO_HIPODROMO LA ZARZUELA,HIPODROMO_HIPÓDROMO DE PINEDA,PISTA_ARENA,PISTA_ARENA-FIBRA,PISTA_HIERBA
0,1_01-01-2017,1.0,3.0,56.5,1.0,1300,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
1,1_01-01-2017,3.0,3.0,56.0,4.0,1300,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
2,1_01-01-2017,6.0,3.0,56.0,5.0,1300,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
3,1_01-01-2017,2.0,3.0,54.5,2.0,1300,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
4,1_01-01-2017,4.0,3.0,54.5,3.0,1300,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7,7_26-10-2014,3.0,4.0,56.0,8.0,1800,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
8,7_26-10-2014,2.0,3.0,55.5,10.0,1800,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
9,7_26-10-2014,4.0,5.0,54.0,1.0,1800,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
10,7_26-10-2014,6.0,4.0,52.0,4.0,1800,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1


In [99]:
#Queremos dividir los datos y que cada id se encuentre completo por lo que utilizamos GroupShuffleSplit

gss = GroupShuffleSplit(train_size=.60, n_splits=1, random_state = 7).split(df, groups=df['ID'])

train_inds, test_inds = next(gss)

train= df.iloc[train_inds]
X_train = train.loc[:, train.columns!='POSICION']
y_train = train['POSICION'].copy()

test= df.iloc[test_inds]

X_test = test.loc[:, train.columns!='POSICION']
y_test = test[['ID','POSICION']].copy()

In [26]:
#Comprobamos que se han dividido correctamente los datos
X_train['ID'].value_counts()
#Como en cada ID como minimo hay cuatro caballos se ha dividido correctamente

3_14-06-2020    20
3_17-11-2013    20
3_30-03-2014    19
3_01-06-2014    19
3_01-12-2013    19
                ..
5_17-06-2020     4
5_24-11-2013     4
3_25-10-2015     4
5_26-03-2021     4
1_12-05-2019     4
Name: ID, Length: 839, dtype: int64

In [27]:
X_test['ID'].value_counts()

3_04-12-2011    20
3_30-09-2012    20
2_30-09-2012    20
3_17-09-2017    20
3_19-10-2014    19
                ..
1_3-09-2016      5
1_04-10-2015     4
5_27-08-2012     4
1_15-11-2015     4
3_26-09-2014     4
Name: ID, Length: 560, dtype: int64

In [28]:
y_train

0    1.0
1    3.0
2    6.0
3    2.0
4    4.0
    ... 
3    5.0
4    3.0
5    7.0
6    2.0
7    4.0
Name: POSICION, Length: 7951, dtype: float64

In [102]:
y_test

,ID,POSICION
0,1_01-08-2013,2.0
1,1_01-08-2013,1.0
2,1_01-08-2013,4.0
3,1_01-08-2013,5.0
4,1_01-08-2013,99.0
...,...,...
7,7_26-10-2014,3.0
8,7_26-10-2014,2.0
9,7_26-10-2014,4.0
10,7_26-10-2014,6.0


In [30]:
#Una vez comprobado que se han dividido correctamente, eliminamos la columna ID
X_train = X_train.drop('ID',axis=1).copy()
X_train

,EDAD,KG,CAJÓN,DISTANCIA,CABALLO_AARASH,CABALLO_AARHUS,CABALLO_AAWAM DE GRINE (MOR),CABALLO_ABANTOS (IRE),CABALLO_ABDEL (FR),CABALLO_ABDEL(FR),...,PREPARADOR_Y. DUREPAIRE,PREPARADOR_Y.DUREPAIRE,HIPODROMO_GRAN HIPODROMO DE ANDALUCIA JAVIER PIÑAR HAFNER,HIPODROMO_HIPODROMO COSTA DEL SOL,HIPODROMO_HIPODROMO DE SAN SEBASTIAN,HIPODROMO_HIPODROMO LA ZARZUELA,HIPODROMO_HIPÓDROMO DE PINEDA,PISTA_ARENA,PISTA_ARENA-FIBRA,PISTA_HIERBA
0,3.0,56.5,1.0,1300,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
1,3.0,56.0,4.0,1300,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
2,3.0,56.0,5.0,1300,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
3,3.0,54.5,2.0,1300,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
4,3.0,54.5,3.0,1300,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3,6.0,58.5,2.0,1600,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
4,7.0,58.5,7.0,1600,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
5,4.0,57.0,6.0,1600,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
6,4.0,57.0,5.0,1600,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1


In [31]:
#Vamos a entrenar el modelo

#Para utilizar XGBoostRanker, debemos indicar el grupo al que debe clasificar
groups = train.groupby('ID').size().to_frame('size')['size'].to_numpy()

#Elegimos los hipermarametros del entrenamiento
clf_xgb=xgb.XGBRanker( 
    booster='gbtree',
    objective='rank:ndcg',
    random_state=7, 
    learning_rate=0.05, 
    max_depth=6, 
    n_estimators=100, 
    subsample=0.5 
    )

clf_xgb.fit(X_train, y_train,group =groups)

XGBRanker(base_score=0.5, booster='gbtree', colsample_bylevel=1,
          colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
          importance_type='gain', interaction_constraints='',
          learning_rate=0.05, max_delta_step=0, max_depth=6, min_child_weight=1,
          missing=nan, monotone_constraints='()', n_estimators=100, n_jobs=4,
          num_parallel_tree=1, objective='rank:ndcg', random_state=7,
          reg_alpha=0, reg_lambda=1, scale_pos_weight=None, subsample=0.5,
          tree_method='exact', validate_parameters=1, verbosity=None)

In [33]:
def predict(model, df):
    return model.predict(df.loc[:, ~df.columns.isin(['ID'])])


predictions = (X_test.groupby('ID')
               .apply(lambda x: predict(clf_xgb, x)))

In [34]:
predictions

ID
1_01-08-2013    [0.47735432, 1.0493529, 0.53872967, 0.7735403,...
1_01-11-2015    [0.12595832, -0.11505503, 0.23754826, 0.511376...
1_02-11-2014    [-0.35401514, 0.012470183, -0.110074624, 0.395...
1_02-12-2012    [-0.67891693, -0.09319937, 0.0030314971, -0.21...
1_03-07-2016    [0.5785989, 0.65466076, 0.23445444, 0.5437027,...
                                      ...                        
5_8-03-2020     [-0.40183955, -0.12037075, -0.3130777, -0.3756...
5_9-04-2017     [0.1764642, -0.10497626, 0.16214013, 0.1698497...
5_9-10-2016     [0.17894088, 0.14177921, 0.15446882, 0.2482155...
6_26-10-2014    [0.012274863, 0.11638886, 0.02943842, 0.087257...
7_26-10-2014    [-0.018594159, -0.17644954, 0.014698514, 0.164...
Length: 560, dtype: object

In [35]:
#La salida de predictions nos indica es que si es un numero alto, este caballo tiene mas posibilidades de ganar sobre el resto
predictions['1_01-08-2013']

array([0.47735432, 1.0493529 , 0.53872967, 0.7735403 , 0.9978258 ,
       0.8636462 , 0.30600488], dtype=float32)

In [36]:
#Comprobamos en el archivo inicial y la prediccion del ganador es correcta
df.loc[df['ID']=='1_01-08-2013']

,ID,POSICION,EDAD,KG,CAJÓN,DISTANCIA,CABALLO_AARASH,CABALLO_AARHUS,CABALLO_AAWAM DE GRINE (MOR),CABALLO_ABANTOS (IRE),...,PREPARADOR_Y. DUREPAIRE,PREPARADOR_Y.DUREPAIRE,HIPODROMO_GRAN HIPODROMO DE ANDALUCIA JAVIER PIÑAR HAFNER,HIPODROMO_HIPODROMO COSTA DEL SOL,HIPODROMO_HIPODROMO DE SAN SEBASTIAN,HIPODROMO_HIPODROMO LA ZARZUELA,HIPODROMO_HIPÓDROMO DE PINEDA,PISTA_ARENA,PISTA_ARENA-FIBRA,PISTA_HIERBA
0,1_01-08-2013,2.0,2.0,56.0,5.0,1200,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
1,1_01-08-2013,1.0,2.0,56.0,3.0,1200,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
2,1_01-08-2013,4.0,2.0,54.5,2.0,1200,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
3,1_01-08-2013,5.0,2.0,54.5,6.0,1200,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
4,1_01-08-2013,99.0,2.0,54.5,4.0,1200,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
5,1_01-08-2013,99.0,2.0,54.5,7.0,1200,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
6,1_01-08-2013,3.0,2.0,54.5,1.0,1200,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0


In [37]:
type(predictions)

pandas.core.series.Series

In [38]:
predictions.index

Index(['1_01-08-2013', '1_01-11-2015', '1_02-11-2014', '1_02-12-2012',
       '1_03-07-2016', '1_03-07-2020', '1_03-10-2014', '1_04-05-2014',
       '1_04-10-2015', '1_05-03-2017',
       ...
       '5_30-12-2020', '5_31-03-2019', '5_31-08-2015', '5_4-07-2019',
       '5_7-07-2016', '5_8-03-2020', '5_9-04-2017', '5_9-10-2016',
       '6_26-10-2014', '7_26-10-2014'],
      dtype='object', name='ID', length=560)

In [95]:
#Convertimos la serie en un dataframe
data_index = predictions.keys().to_list()
data=[]

for elem in predictions:
    data.append(dict(zip(range(len(elem)),elem)))
    
predictions_df = pd.DataFrame(data, index = data_index)

In [96]:
predictions_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
1_01-08-2013,0.477354,1.049353,0.538730,0.773540,0.997826,0.863646,0.306005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1_01-11-2015,0.125958,-0.115055,0.237548,0.511377,0.790161,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1_02-11-2014,-0.354015,0.012470,-0.110075,0.395446,-0.011052,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1_02-12-2012,-0.678917,-0.093199,0.003031,-0.216507,0.536762,0.009554,0.056959,-0.014981,0.070718,0.125411,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1_03-07-2016,0.578599,0.654661,0.234454,0.543703,0.132745,0.420282,1.015805,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5_8-03-2020,-0.401840,-0.120371,-0.313078,-0.375643,0.071877,0.040823,0.386942,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5_9-04-2017,0.176464,-0.104976,0.162140,0.169850,0.202638,0.203231,-0.135244,-0.358974,0.181739,0.152101,0.438759,0.230853,0.279350,0.063676,0.182939,0.186531,0.159655,NaN,NaN,NaN
5_9-10-2016,0.178941,0.141779,0.154469,0.248216,0.058435,0.022161,0.480046,0.467123,0.504747,-0.397250,-0.044061,0.231062,0.433889,0.143860,0.905035,NaN,NaN,NaN,NaN,NaN
6_26-10-2014,0.012275,0.116389,0.029438,0.087257,0.254851,0.821480,0.822292,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [97]:
#Convertir los datos en la posicion de la prediccion
data_index = predictions.keys().to_list()
data=[]

for elem in data_index:
    data_poss = predictions_df.sort_values(by=elem, axis=1, ascending = False).loc[elem].dropna()
    data_poss_pred = data_poss.keys().to_list()
    data.append(data_poss_pred)
    
predictions_df = pd.DataFrame(data, index = data_index)
predictions_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
1_01-08-2013,1,4,5,3,2.0,0.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1_01-11-2015,4,3,2,0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1_02-11-2014,3,1,4,2,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1_02-12-2012,4,9,8,6,5.0,2.0,7.0,1.0,3.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1_03-07-2016,6,1,0,3,5.0,2.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5_8-03-2020,6,4,5,1,2.0,3.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5_9-04-2017,10,12,11,5,4.0,15.0,14.0,8.0,0.0,3.0,2.0,16.0,9.0,13.0,1.0,6.0,7.0,NaN,NaN,NaN
5_9-10-2016,14,8,6,7,12.0,3.0,11.0,0.0,2.0,13.0,1.0,4.0,5.0,10.0,9.0,NaN,NaN,NaN,NaN,NaN
6_26-10-2014,6,5,4,1,3.0,2.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [72]:
predictions1= pd.DataFrame(predictions, columns=['poss_pred'])

In [ ]:
type(predictions1)

In [ ]:
predictions1['poss_pred']

In [ ]:
predictions1.columns

In [ ]:
predictions1.dtypes

In [ ]:
predictions1['1']= predictions1['poss_pred'].str.split(',', 1)

In [ ]:
predictions1

In [ ]:
predictions1['ID'], predictions1['poss_pred'] = predictions1.split(' ', 1)

In [ ]:
predictions1 = pd.DataFrame(predictions1.ID.str.split(' ',1).tolist(),
                                   columns = ['ID','POSICION'])

In [ ]:
predictions1['ID']=predictions1['ID'].map(lambda x:x.split(' ')[0])
predictions1

In [ ]:
for i in predictions1.values:
    i_stitched = pd.concat(i, axis=1)

In [ ]:
for i in predictions1.values:
    i.char.split(sep=',')
    print(i)

In [ ]:
for i in predictions1.values:
    for j in i:
        print(j)